#  set autocomplete

In [1]:
%config IPCompleter.greedy=True

# Eager Mode Mnist

In [18]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
from tensorflow.keras import Model 
from tensorflow.keras.layers import Conv2D, Flatten, Dense
import numpy as np

In [20]:
print(tf.__version__=='2.0.0-alpha0')

True


mnist image : 28,28 

In [3]:
mnist = tf.keras.datasets.mnist

# Create Model

In [4]:
from tensorflow.keras import layers

class Mnist(tf.keras.Sequential):
    def __init__(self):
        super(Mnist, self).__init__()
        self.conv1 = Conv2D(16,[3,3], activation='relu')
        self.conv2 = Conv2D(16,[3,3], activation='relu')
        self.conv3 = Conv2D(16,[3,3], activation='relu')
        self.flat  = Flatten()
        self.dense = Dense(10, activation='softmax')
    
    def __call__(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.flat(x)
        x = self.dense(x)
        return x

# model = Mnist()
# print(model)
# random_state = 42
# test1 = tf.cast(tf.random.normal([1,28,28,1]), tf.float32)
# test2 = tf.cast(tf.random.normal([1,28,28,1]), tf.float64)
# print(model(test1))
# print(model(test2))

# Load Data 

In [12]:
# Fetch and format the mnist data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
x_train = x_train / 255.0
x_test = x_test / 255.0

train_dataset = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

print(f"trainset size : {train_dataset}")
print(f"testset  size : {test_dataset}")
# dataset = dataset.shuffle(1000).batch(32)

trainset size : <BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float64, tf.uint8)>
testset  size : <BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float64, tf.uint8)>


In [13]:
model = Mnist()

# Define Optimizer

In [14]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

# Train

In [15]:
@tf.function
def train_step(model, image, label):
  with tf.GradientTape() as tape:
    predictions = model(image)
    loss = loss_object(label, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(label, predictions)

In [16]:
epochs=2

for epoch in range(epochs):
    for image, label in train_dataset:
        train_step(model, image, label)    
    print(f"epoch:{epoch}, loss:{train_loss.result()}, acc:{train_accuracy.result()*100}")        

epoch:0, loss:0.1466822624206543, acc:95.63999938964844
epoch:1, loss:0.09928521513938904, acc:97.02083587646484
